In [1]:
import requests
import pandas as pd
from sklearn.cluster import KMeans
import json
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.distance import geodesic

In [4]:
df=pd.read_html("https://en.wikipedia.org/wiki/List_of_industrial_cities_in_India")[0]
df.drop(df.columns[[0, 3]], axis = 1, inplace = True) 
df.to_csv("industrial_cities.csv")

In [5]:
df=pd.read_csv("industrial_cities.csv")
lat=[]
lon=[]
for x in df.Place:
    location=None
    
    while location is None:
        geolocator=Nominatim(user_agent="industries")
        location=geolocator.geocode('{}, India'.format(x))
        x=str(x).split(" ")[:-1]
    lat.append(location.latitude)
    print(location)
    print(location.latitude)
    lon.append(location.longitude)
    print(location.longitude)

Coimbatore, Coimbatore North, Coimbatore District, Tamil Nadu, 641001, India
11.0018115
76.9628425
Rudrapur, Kichha, Udham Singh Nagar, Uttarakhand, 263153, India
28.974744
79.3963951
Bhilai, Durg Tahsil, Durg, Chhattisgarh, 490023, India
21.2120677
81.3732849
Raigarh, Chhattisgarh, India
22.5
83.5
Navi Mumbai, Thane, Maharashtra, 400706, India
19.0308262
73.0198537
Aluva, Ernakulam district, Kerala, India
10.17039815
76.38839448844891
Vijayawada, Vijayawada (Urban), Krishna, Andhra Pradesh, 520001, India
16.5087586
80.6185102
Firozabad, Uttar Pradesh, India
27.17736635
78.38991197951182
Panna, Madhya Pradesh, 488058, India
24.5
80.25
Panipat, Haryana, 132100, India
29.3912753
76.9771675
Channapatna, Railway Station Road, Channapattana, Channapattana taluk, Ramanagara district, Karnataka, 571500, India
12.6593727
77.2038931
Varanasi, Uttar Pradesh, 221001, India
25.3356491
83.0076292
Moradabad, Uttar Pradesh, India
28.8638424
78.80577833091104
Bareilly, Uttar Pradesh, India
28.457876
7

Lucknow, Sadar, Lucknow, Uttar Pradesh, 226019, India
26.8381
80.9346001
Moradabad, Uttar Pradesh, India
28.8638424
78.80577833091104
Mysuru, Mysuru taluk, Mysuru district, Karnataka, 570001, India
12.3051828
76.6553609
Raghuveer Hospital, Raebareli, SH13, Lalganj, Rae Bareilly, Uttar Pradesh, 229001, India
26.1639279
80.9699236
Nandambakkam, Ward 158, Zone 12 Alandur, Chennai, Alandur, Tamil Nadu, 600016, India
13.0166484
80.1911885
Nangal, Nangal Tahsil, Rupnagar, Punjab, 140124, India
31.3837484
76.3754353
Nepanagar, Nepanagar Tahsil, Burhanpur, Madhya Pradesh, India
21.4641072
76.3968819
Neyveli, Virudhachalam, Cuddalore District, Tamil Nadu, 607803, India
11.535987
79.4824392
Udhagamandalam, The Nilgiris District, Tamil Nadu, 643001, India
11.4126769
76.7030504
Perambur, Ward 70, Zone 6 Thiru. Vi. Ka. Nagar, Chennai, Chennai District, Tamil Nadu, 600011, India
13.1121242
80.245022
Pinjore, Kalka, Panchkula, Haryana, 134102, India
30.8042411
76.9149498
Rupnarayanpur, Kharagpur-I, P

In [6]:
df["latitude"]=lat
df["longitude"]=lon
df

,Unnamed: 0,Place,State,latitude,longitude
0,0,Coimbatore,Tamil Nadu,11.001812,76.962842
1,1,Rudrapur,Uttarakhand,28.974744,79.396395
2,2,Bhilai,Chhattisgarh,21.212068,81.373285
3,3,Raigarh,Chhattisgarh,22.500000,83.500000
4,4,Navi Mumbai,Maharashtra,19.030826,73.019854
...,...,...,...,...,...
124,124,Sivakasi,Tamil Nadu,9.466990,77.775979
125,125,Udhana,Gujarat,21.165091,72.839774
126,126,Worli,Maharastra,19.011696,72.818070
127,127,Zaina Kot,Jammu and Kashmir,10.108135,76.358977


In [7]:
map=folium.Map(location=[20.593684,78.96288],zoom_start=5)
for lat, lng, city in zip(df['latitude'], df['longitude'],df['Place']):
    label = '{}, India'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#ffa849',
        fill=True,
        fill_color='#4a2b13',
        fill_opacity=0.7,
        parse_html=False).add_to(map)
map

In [8]:
CLIENT_ID='HBQ1QJTILHSLNN0PQXFOY0MTA5QYSOCFG0GYKOJYDFBZTYSH'
CLIENT_SECRET='AZXGBWSNOHWWDXWTXKRW5BJV2IGCA1OAUFNA3VNSTXNWRV2G'
LIMIT=100
VERSION='20180604'

In [9]:
def getNearbyVenues(names, latitudes, longitudes, cat,radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            cat)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Place', 
                  'Place Latitude', 
                  'Place Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
categories='4eb1bea83b7b6f98df247e06'
venues = getNearbyVenues(names=df['Place'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude'],
                                    cat=categories)

Coimbatore
Rudrapur
Bhilai
Raigarh
Navi Mumbai
Aluva
Vijayawada
Firozabad
Panna
Panipat
Channapatna
Varanasi
Moradabad
Bareilly
Jamshedpur
Tiruppur
Rajahmundry
Bokaro Steel City
Kolkata
Durgapur
Kharagpur
Hosdurg
Haldia
Indore
Pithampur
Belagavi
Dibrugarh
Palakkad
Delhi
Noida
Kannur
Salem
Sivakasi
Kanpur
Rajkot
Kochi
Peenya
Surat
Rourkela
Visakhapatnam
Angul
Ahmedabad
Gandhinagar
Jamnagar
Vadodara
Bharuch Ankleshwar
Ludhiana
Pimpri-Chinchwad
Mumbai
Bangalore
Thiruvananthapuram
Thrissur
Gajraula
Unnao
Sri City
Margao
Kunnamkulam
Darjeeling
Nashik
Chennai
Hosur
Haridwar
Faridabad
Mathura
Gurugram
Baddi
Calicut
Pune
Aurangabad
Nagpur
Mirzapur
Perambra
Muzaffarnagar
Meerut
Saharanpur
Bulandshahr
Ichalkaranji
Muvattupuzha
Aligarh
Anjar
Bardoli
Chittaranjan
Hyderabad
Jharia
Morbi
Nepanagar
Kollam
Tiruvalla
Kottayam
Idukki
Alappuzha
Cherthala
Alwaye
Ambattur
Ambarnath
Amritsar
Avadi
Ankleshwar
Bhadravati
Cochin
Dindigul
Ennore
Guntur
Kagithapuram
Kalpakkam
Kanpur
Kollegal
Kolar
Lucknow
Morada

In [14]:
venues

In [26]:
categories1='56aa371be4b08b9a8d5734d7'
venues1 = getNearbyVenues(names=df['Place'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude'],
                                    cat=categories1)

Coimbatore
Rudrapur
Bhilai
Raigarh
Navi Mumbai
Aluva
Vijayawada
Firozabad
Panna
Panipat
Channapatna
Varanasi
Moradabad
Bareilly
Jamshedpur
Tiruppur
Rajahmundry
Bokaro Steel City
Kolkata
Durgapur
Kharagpur
Hosdurg
Haldia
Indore
Pithampur
Belagavi
Dibrugarh
Palakkad
Delhi
Noida
Kannur
Salem
Sivakasi
Kanpur
Rajkot
Kochi
Peenya
Surat
Rourkela
Visakhapatnam
Angul
Ahmedabad
Gandhinagar
Jamnagar
Vadodara
Bharuch Ankleshwar
Ludhiana
Pimpri-Chinchwad
Mumbai
Bangalore
Thiruvananthapuram
Thrissur
Gajraula
Unnao
Sri City
Margao
Kunnamkulam
Darjeeling
Nashik
Chennai
Hosur
Haridwar
Faridabad
Mathura
Gurugram
Baddi
Calicut
Pune
Aurangabad
Nagpur
Mirzapur
Perambra
Muzaffarnagar
Meerut
Saharanpur
Bulandshahr
Ichalkaranji
Muvattupuzha
Aligarh
Anjar
Bardoli
Chittaranjan
Hyderabad
Jharia
Morbi
Nepanagar
Kollam
Tiruvalla
Kottayam
Idukki
Alappuzha
Cherthala
Alwaye
Ambattur
Ambarnath
Amritsar
Avadi
Ankleshwar
Bhadravati
Cochin
Dindigul
Ennore
Guntur
Kagithapuram
Kalpakkam
Kanpur
Kollegal
Kolar
Lucknow
Morada

In [28]:
target=venues.append(venues1)
venues1

,Place,Place Latitude,Place Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Coimbatore,11.001812,76.962842,Luxen India,11.012150,76.964637,Industrial Estate
1,Delhi,28.651718,77.221939,Singer India Limited,28.644498,77.225819,Industrial Estate
2,Noida,28.535633,77.391073,Indostar TMT,28.528357,77.388124,Industrial Estate
3,Peenya,13.032942,77.527325,Essentra India Private Limited,13.040051,77.527860,Industrial Estate
4,Peenya,13.032942,77.527325,Ingwenya Mineral Tech Pvt Ltd,13.026445,77.519245,Industrial Estate
5,Peenya,13.032942,77.527325,Steer,13.023350,77.514674,Industrial Estate
6,Peenya,13.032942,77.527325,"Industrial Estate,peenya",13.031025,77.520561,Factory
7,Ahmedabad,23.021624,72.579707,Air N Gas Process Technologies,23.021730,72.580394,Industrial Estate
8,Ahmedabad,23.021624,72.579707,Toyota Forms India Pvt Ltd.,23.023184,72.580705,Industrial Estate
9,Ahmedabad,23.021624,72.579707,Dynamic Engineers,23.021280,72.576327,Industrial Estate


In [24]:
categories2='4bf58dd8d48988d175941735'
venues2 = getNearbyVenues(names=df['Place'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude'],
                                    cat=categories2)

Coimbatore
Rudrapur
Bhilai
Raigarh
Navi Mumbai
Aluva
Vijayawada
Firozabad
Panna
Panipat
Channapatna
Varanasi
Moradabad
Bareilly
Jamshedpur
Tiruppur
Rajahmundry
Bokaro Steel City
Kolkata
Durgapur
Kharagpur
Hosdurg
Haldia
Indore
Pithampur
Belagavi
Dibrugarh
Palakkad
Delhi
Noida
Kannur
Salem
Sivakasi
Kanpur
Rajkot
Kochi
Peenya
Surat
Rourkela
Visakhapatnam
Angul
Ahmedabad
Gandhinagar
Jamnagar
Vadodara
Bharuch Ankleshwar
Ludhiana
Pimpri-Chinchwad
Mumbai
Bangalore
Thiruvananthapuram
Thrissur
Gajraula
Unnao
Sri City
Margao
Kunnamkulam
Darjeeling
Nashik
Chennai
Hosur
Haridwar
Faridabad
Mathura
Gurugram
Baddi
Calicut
Pune
Aurangabad
Nagpur
Mirzapur
Perambra
Muzaffarnagar
Meerut
Saharanpur
Bulandshahr
Ichalkaranji
Muvattupuzha
Aligarh
Anjar
Bardoli
Chittaranjan
Hyderabad
Jharia
Morbi
Nepanagar
Kollam
Tiruvalla
Kottayam
Idukki
Alappuzha
Cherthala
Alwaye
Ambattur
Ambarnath
Amritsar
Avadi
Ankleshwar
Bhadravati
Cochin
Dindigul
Ennore
Guntur
Kagithapuram
Kalpakkam
Kanpur
Kollegal
Kolar
Lucknow
Morada

In [29]:
target=target.append(venues2)
venues2

,Place,Place Latitude,Place Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Coimbatore,11.001812,76.962842,Posh,11.005329,76.951314,Gym / Fitness Center
1,Coimbatore,11.001812,76.962842,Golds Gym,11.005079,76.951123,Gym
2,Coimbatore,11.001812,76.962842,F45 Training Coimbatore Racecourse,11.004293,76.979235,Gym
3,Rudrapur,28.974744,79.396395,fitness world,28.979806,79.379765,Gym
4,Navi Mumbai,19.030826,73.019854,Chavana Spa,19.032892,73.029589,Gym / Fitness Center
...,...,...,...,...,...,...,...
229,Kolhapur,16.702841,74.240533,parth gym,16.710168,74.234874,Gym
230,Kolhapur,16.702841,74.240533,Gold's Gym,16.711383,74.245034,Gym
231,Kolhapur,16.702841,74.240533,Talwalkar's prime fitness,16.712305,74.244536,Gym
232,Kolhapur,16.702841,74.240533,Parsana Gym,16.704797,74.243706,Yoga Studio


In [36]:
target
target.to_csv("target.csv")

In [37]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [39]:
writer = pd.ExcelWriter('target.xlsx')
target.to_excel(writer)
writer.save()